In [1]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
import torch
from scipy.ndimage import gaussian_filter
from scipy import ndimage

X = []
# Define a function to extract ROIs around microbleed locations
def extract_roi(swi_data, coordinates, roi_size):
    rois = []
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]
        rois.append(roi)
    return rois

# Define a function to compute Hessian matrix eigenvalues
def compute_hessian_eigenvalues(roi):
    # Apply Gaussian filter to the ROI for smoothing
    smoothed_roi = gaussian_filter(roi, sigma=1)
    
    # Compute gradients along x, y, and z directions using Sobel filters
    gradient_x = ndimage.sobel(smoothed_roi, axis=0)
    gradient_y = ndimage.sobel(smoothed_roi, axis=1)
    gradient_z = ndimage.sobel(smoothed_roi, axis=2)
    
    # Compute second derivatives along x, y, and z directions
    dxx = ndimage.sobel(gradient_x, axis=0)
    dyy = ndimage.sobel(gradient_y, axis=1)
    dzz = ndimage.sobel(gradient_z, axis=2)
    dxy = ndimage.sobel(gradient_x, axis=1)
    dxz = ndimage.sobel(gradient_x, axis=2)
    dyz = ndimage.sobel(gradient_y, axis=2)
    
    # Construct the Hessian matrix
    Hessian_matrix = np.array([
        [dxx.mean(), dxy.mean(), dxz.mean()],
        [dxy.mean(), dyy.mean(), dyz.mean()],
        [dxz.mean(), dyz.mean(), dzz.mean()]
    ])
    
    # Compute eigenvalues and eigenvectors of the Hessian matrix
    eigenvalues, eigenvectors = np.linalg.eig(Hessian_matrix)
    
    return eigenvalues, eigenvectors

# Define a function to compute Hessian shape features
def compute_hessian_shape_features(eigenvalues):
    if len(eigenvalues) != 3:
        # If the number of eigenvalues is not 3, return default values or handle the case accordingly
        print("Invalid number of eigenvalues for computing features")
        return 0, 0, 0  # Return default values or handle differently if needed
    
    # Sphericalness feature
    f_sphere = abs(eigenvalues[0]) / np.sqrt(abs(eigenvalues[1] * eigenvalues[2]))
    
    # Largest cross-section feature
    f_lc = abs(eigenvalues[1]) / abs(eigenvalues[2])
    
    # Fractional anisotropy feature
    f_fa = np.sqrt(0.5) * np.sqrt((eigenvalues[0] - eigenvalues[1])**2 + 
                                  (eigenvalues[1] - eigenvalues[2])**2 + 
                                  (eigenvalues[0] - eigenvalues[2])**2) / np.sqrt((eigenvalues[0]**2 + eigenvalues[1]**2 + eigenvalues[2]**2))
    
    return f_sphere, f_lc, f_fa

# Load the Excel file containing microbleed coordinates
excel_file = 'micro-location.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'rCMB'  # Replace 'path_to_rCMB_folder' with your folder path

# Iterate through rows to load NIFTI files and extract microbleed features
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        #print(f"NIFTI Filename: {nifti_filename}")
        #print(f"Microbleed Coordinates: {microbleed_coordinates}")
        #print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Extract ROIs around microbleed locations
        microbleed_rois = extract_roi(swi_data, microbleed_coordinates, roi_size)

        # Compute Hessian matrix and eigenvalues for each ROI
        eigenvalues_list = []
        for roi in microbleed_rois:
            eigenvalues = compute_hessian_eigenvalues(roi)
            eigenvalues_list.append(eigenvalues)

        # Compute Hessian shape features for each microbleed
        hessian_shape_features_microbleed = []
        for i, eigenvals_tuple in enumerate(eigenvalues_list):
            eigenvals = eigenvals_tuple[0]  # Unpack the nested structure
            if len(eigenvals) != 3:
                # Print the invalid eigenvalues and their count for inspection
                print(f"Invalid eigenvalues at index {i+1}: {eigenvals} (Count: {len(eigenvals)})")
                continue
            
            f_sphere, f_lc, f_fa = compute_hessian_shape_features(eigenvals)
            hessian_shape_features_microbleed.append((f_sphere, f_lc, f_fa))

        # Append the computed features to the main feature list (X)
        X.extend(hessian_shape_features_microbleed)

# Convert the list of microbleed features to a NumPy array
X1 = np.array(X)

# Display the shape of the resulting feature array
print(f"Shape of X (microbleed features): {X1.shape}")

C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\1803081685.py:99: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\1803081685.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\1803081685.py:101: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(r

Shape of X (microbleed features): (1666, 3)


In [2]:
# Assuming X contains the microbleed features

# Determine the number of microbleeds in X
num_microbleeds = X1.shape[0]

# Set the labels for microbleeds as 1
Y1 = np.ones(num_microbleeds)

# Display the shape of the label array
print(f"Shape of Y1 (labels): {Y1.shape}")

# Display a few labels to confirm
print(Y1)  

Shape of Y1 (labels): (1666,)
[1. 1. 1. ... 1. 1. 1.]


In [3]:
import pandas as pd
import os
import nibabel as nib
import numpy as np
from scipy.ndimage import gaussian_filter
from scipy import ndimage



X = []
# Define a function to extract ROIs around microbleed locations
def extract_roi(swi_data, coordinates, roi_size):
    rois = []
    for coord in coordinates:
        x, y, z = coord
        # Ensure the ROI stays within the image bounds
        x_start = max(0, x - roi_size // 2)
        x_end = min(swi_data.shape[0], x + (roi_size + 1) // 2)
        y_start = max(0, y - roi_size // 2)
        y_end = min(swi_data.shape[1], y + (roi_size + 1) // 2)
        z_start = max(0, z - roi_size // 2)
        z_end = min(swi_data.shape[2], z + (roi_size + 1) // 2)
        
        # Extract the ROI from the image data
        roi = swi_data[x_start:x_end, y_start:y_end, z_start:z_end]
        rois.append(roi)
    return rois

# Define a function to compute Hessian matrix eigenvalues
def compute_hessian_eigenvalues(roi):
    # Apply Gaussian filter to the ROI for smoothing
    smoothed_roi = gaussian_filter(roi, sigma=1)
    
    # Compute gradients along x, y, and z directions using Sobel filters
    gradient_x = ndimage.sobel(smoothed_roi, axis=0)
    gradient_y = ndimage.sobel(smoothed_roi, axis=1)
    gradient_z = ndimage.sobel(smoothed_roi, axis=2)
    
    # Compute second derivatives along x, y, and z directions
    dxx = ndimage.sobel(gradient_x, axis=0)
    dyy = ndimage.sobel(gradient_y, axis=1)
    dzz = ndimage.sobel(gradient_z, axis=2)
    dxy = ndimage.sobel(gradient_x, axis=1)
    dxz = ndimage.sobel(gradient_x, axis=2)
    dyz = ndimage.sobel(gradient_y, axis=2)
    
    # Construct the Hessian matrix
    Hessian_matrix = np.array([
        [dxx.mean(), dxy.mean(), dxz.mean()],
        [dxy.mean(), dyy.mean(), dyz.mean()],
        [dxz.mean(), dyz.mean(), dzz.mean()]
    ])
    
    # Compute eigenvalues and eigenvectors of the Hessian matrix
    eigenvalues, eigenvectors = np.linalg.eig(Hessian_matrix)
    
    return eigenvalues, eigenvectors

# Define a function to compute Hessian shape features
def compute_hessian_shape_features(eigenvalues):
    if len(eigenvalues) != 3:
        # If the number of eigenvalues is not 3, return default values or handle the case accordingly
        print("Invalid number of eigenvalues for computing features")
        return 0, 0, 0  # Return default values or handle differently if needed
    
    # Sphericalness feature
    f_sphere = abs(eigenvalues[0]) / np.sqrt(abs(eigenvalues[1] * eigenvalues[2]))
    
    # Largest cross-section feature
    f_lc = abs(eigenvalues[1]) / abs(eigenvalues[2])
    
    # Fractional anisotropy feature
    f_fa = np.sqrt(0.5) * np.sqrt((eigenvalues[0] - eigenvalues[1])**2 + 
                                  (eigenvalues[1] - eigenvalues[2])**2 + 
                                  (eigenvalues[0] - eigenvalues[2])**2) / np.sqrt((eigenvalues[0]**2 + eigenvalues[1]**2 + eigenvalues[2]**2))
    
    return f_sphere, f_lc, f_fa

# Load the Excel file containing microbleed coordinates
excel_file = 'NoCMB.xlsx'  # Replace with your Excel file path
df = pd.read_excel(excel_file)

# Path to the folder containing the NIFTI images
folder_path = 'noncmb'  # Replace 'path_to_rCMB_folder' with your folder path

# Iterate through rows to load NIFTI files and extract microbleed features
for index, row in df.iterrows():
    nifti_filename = row['NIFTI']  # Assuming 'NIFTI' is the column name containing the NIFTI file names
    nifti_path = os.path.join(folder_path, nifti_filename)

    # Check if the NIFTI file exists in the folder
    if os.path.exists(nifti_path):
        swi_img = nib.load(nifti_path)
        swi_data = swi_img.get_fdata()

        # Extract microbleed coordinates for the current NIFTI file
        microbleed_coordinates = []
        for i in range(1, len(row), 3):
            try:
                if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
                    x = int(row[i])
                    y = int(row[i+1])
                    z = int(row[i+2])
                    microbleed_coordinates.append((x, y, z))
            except (ValueError, TypeError):
                pass  # Skip non-numeric or empty values

        # Process microbleed coordinates for the current NIFTI file
        #print(f"NIFTI Filename: {nifti_filename}")
        #print(f"Microbleed Coordinates: {microbleed_coordinates}")
        #print(f"SWI Image Shape: {swi_data.shape}")
        
        # Define the size of the ROI around each microbleed
        roi_size = 10  # Adjust this according to your desired size

        # Extract ROIs around microbleed locations
        microbleed_rois = extract_roi(swi_data, microbleed_coordinates, roi_size)

        # Compute Hessian matrix and eigenvalues for each ROI
        eigenvalues_list = []
        for roi in microbleed_rois:
            eigenvalues = compute_hessian_eigenvalues(roi)
            eigenvalues_list.append(eigenvalues)

        # Compute Hessian shape features for each microbleed
        hessian_shape_features_microbleed = []
        for i, eigenvals_tuple in enumerate(eigenvalues_list):
            eigenvals = eigenvals_tuple[0]  # Unpack the nested structure
            if len(eigenvals) != 3:
                # Print the invalid eigenvalues and their count for inspection
                print(f"Invalid eigenvalues at index {i+1}: {eigenvals} (Count: {len(eigenvals)})")
                continue
            
            f_sphere, f_lc, f_fa = compute_hessian_shape_features(eigenvals)
            hessian_shape_features_microbleed.append((f_sphere, f_lc, f_fa))

        # Append the computed features to the main feature list (X)
        X.extend(hessian_shape_features_microbleed)

# Convert the list of microbleed features to a NumPy array
X0 = np.array(X)

# Display the shape of the resulting feature array
print(f"Shape of X (microbleed features): {X0.shape}")

C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\708017441.py:100: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if pd.notnull(row[i]) and pd.notnull(row[i+1]) and pd.notnull(row[i+2]):
C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\708017441.py:101: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = int(row[i])
C:\Users\Dell\AppData\Local\Temp\ipykernel_2212\708017441.py:102: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  y = int(row

Shape of X (microbleed features): (300, 3)


In [4]:
# Assuming X contains the microbleed features

# Determine the number of microbleeds in X
num_microbleeds = X0.shape[0]

# Set the labels for microbleeds as 1
Y0 = np.zeros(num_microbleeds)

# Display the shape of the label array
print(f"Shape of Y0 (labels): {Y0.shape}")

# Display a few labels to confirm
print(Y0)  # Displaying the first 10 labels as an example


Shape of Y0 (labels): (300,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [5]:
X = np.vstack((X1,X0))

print(X.shape)
print("Length of dataset:", len(X))



# Create corresponding labels 'y' (1 for microbleeds, 0 for non-microbleeds)
Y = np.hstack((Y1,Y0))

print(Y.shape)

(1966, 3)
Length of dataset: 1966
(1966,)


In [6]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import numpy as np

# Function for Layer 1 with LOOCV
def layer1(X_train, y_train, device):
    X_tensor = torch.tensor(StandardScaler().fit_transform(X_train), dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_train, dtype=torch.long).to(device)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_tensor.cpu().numpy(), y_tensor.cpu().numpy())

    y_pred = clf.predict(X_tensor.cpu().numpy())

    positively_predicted_samples1 = X_train[y_pred == y_train]
    actual_labels_positively_predicted1 = y_train[y_pred == y_train]

    return positively_predicted_samples1, actual_labels_positively_predicted1

# Function for Layer 2 with LOOCV
def layer2(X_layer1, y_layer1, device):
    X_tensor = torch.tensor(StandardScaler().fit_transform(X_layer1), dtype=torch.float32).to(device)
    y_tensor = torch.tensor(y_layer1, dtype=torch.long).to(device)

    clf = RandomForestClassifier(random_state=42)
    clf.fit(X_tensor.cpu().numpy(), y_tensor.cpu().numpy())

    y_pred = clf.predict(X_tensor.cpu().numpy())

    positively_predicted_samples2 = X_layer1[y_pred == y_layer1]
    actual_labels_positively_predicted2 = y_layer1[y_pred == y_layer1]

    return positively_predicted_samples2, actual_labels_positively_predicted2, clf

# Assuming X and Y are your original datasets
X_tensor = torch.tensor(X, dtype=torch.float32)
Y_tensor = torch.tensor(Y, dtype=torch.long)

final_layer_accuracies = []
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_tensor = X_tensor.to(device)
Y_tensor = Y_tensor.to(device)

# Convert data to PyTorch DataLoader for efficient processing on GPU
dataset = TensorDataset(X_tensor, Y_tensor)
loader = DataLoader(dataset, batch_size=1, shuffle=False)

for test_index, (X_train, Y_train) in enumerate(loader):
    X_train = X_tensor[torch.arange(len(X_tensor)) != test_index]
    Y_train = Y_tensor[torch.arange(len(Y_tensor)) != test_index]

    positively_predicted_samples1, actual_labels_positively_predicted1 = layer1(X_train.cpu().numpy(), Y_train.cpu().numpy(), device)

    positively_predicted_samples2, actual_labels_positively_predicted2, clf = layer2(positively_predicted_samples1, actual_labels_positively_predicted1, device)

    X_final_layer = positively_predicted_samples2
    Y_final_layer = actual_labels_positively_predicted2

    X_test_tensor = X_tensor[test_index].reshape(1, -1)
    y_pred_final_layer = clf.predict(X_test_tensor.cpu().numpy())

    accuracy_final_layer = accuracy_score([Y[test_index]], y_pred_final_layer)

    final_layer_accuracies.append(accuracy_final_layer)

    print(f"Iteration {test_index + 1}:")
    print(f"Test Data: {X_tensor[test_index]}")
    print(f"Actual Label: {Y_tensor[test_index]}")
    print(f"Predicted Label (Final Layer): {y_pred_final_layer}")
    print(f"Accuracy (Final Layer): {accuracy_final_layer}")
    print()

mean_accuracy_final_layer = np.mean(final_layer_accuracies)
print("Mean Accuracy (Final Layer):", mean_accuracy_final_layer)


Iteration 1:
Test Data: tensor([1.3277, 0.0595, 1.1526], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 2:
Test Data: tensor([3.5165, 0.0687, 1.2224], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 3:
Test Data: tensor([1.2413, 0.2410, 1.1664], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 4:
Test Data: tensor([7.2734, 2.3604, 0.9499], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 5:
Test Data: tensor([7.4240, 5.9142, 1.1232], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 6:
Test Data: tensor([0.9906, 2.3244, 1.1448], device='cuda:0')
Actual Label: 1
Predicted Label (Final Layer): [1]
Accuracy (Final Layer): 1.0

Iteration 7:
Test Data: tensor([ 4.6111, 14.5821,  1.2184], device='cuda:0')